In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
main_url =  'http://www.volkslauf-ergebnisse.de/'

response = requests.get(main_url)
soup = BeautifulSoup(response.content, features="lxml")

In [10]:
# how many pages are there full of runs?
nav = soup.find('div', class_='navigation')
max_page = max([int(a.text) for a in nav.find_all('a')])

pages = [main_url]
for i in range(2, max_page+1):
    url = f'http://www.volkslauf-ergebnisse.de/{i}/date/'
    pages.append(url)
pages[:5]

['http://www.volkslauf-ergebnisse.de/',
 'http://www.volkslauf-ergebnisse.de/2/date/',
 'http://www.volkslauf-ergebnisse.de/3/date/',
 'http://www.volkslauf-ergebnisse.de/4/date/',
 'http://www.volkslauf-ergebnisse.de/5/date/']

* name of run
* kilometres
* number of runners
* city
* postal_code
* street
* date of run

In [6]:
run_urls = []
page = pages[0]
response = requests.get(page)
soup = BeautifulSoup(response.content, features="lxml")

def extract_run_urls(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, features="lxml")
    run_urls = []
    for run in soup.find_all('div', class_='lauf_eintrag'):
        urls = [main_url + a['href'] for a in run.find_all('a', href=True) if a['href'].startswith('/list')]
        run_urls += urls 
    run_urls
    return run_urls

extract_run_urls(page_url=pages[0])[:5]

['http://www.volkslauf-ergebnisse.de//list/35. Hördter Auwaldlauf/1305/',
 'http://www.volkslauf-ergebnisse.de//list/35. Hördter Auwaldlauf/1306/',
 'http://www.volkslauf-ergebnisse.de//list/35. Hördter Auwaldlauf/1307/',
 'http://www.volkslauf-ergebnisse.de//list/Flugplatzserie 2020/1304/',
 'http://www.volkslauf-ergebnisse.de//list/Flugplatzserie 2020/1299/']

In [7]:
def extract_name(soup, remove_digits=False):
    name = soup.find('h1', class_='topic').text
    name = name.strip()

    if remove_digits:
            # remove digits in front of string
            # Example: "35. Hördter Waldlauf" -> Hördter Waldlauf
            name = re.sub(r'\d*\.', '', name).strip()
    return name

def extract_infos(soup):
    location, date, distance, n_runners = soup.find('div', class_='sub_text').text.split(' | ')
    n_runners = [int(n) for n in re.findall(r'\d+', n_runners)][0]
    distance = float(distance.replace(' km', ''))
    return location, date, distance, n_runners

In [11]:
import re

for page in pages[:1]:
        data = []
        runs = extract_run_urls(page)
        for run in runs:
                response = requests.get(run)
                soup = BeautifulSoup(response.content, features="lxml")

                name = extract_name(soup)
                location, date, distance, n_runners = extract_infos(soup)
                row = {
                        'name_of_run': name,
                        'location': location,
                        'date': date,
                        'distance_km': distance,
                        'n_runners': n_runners
                }
                data.append(row)
        df = pd.DataFrame(data)
        # df.to_csv("data/volkslaufergebnisse_runs.csv", mode='a', index=False, encoding="utf-8-sig")
df
    

,name_of_run,location,date,distance_km,n_runners
0,35. Hördter Auwaldlauf,"76771 Hördt, Deutschland",01.03.2020,5.00,86
1,35. Hördter Auwaldlauf,"76771 Hördt, Deutschland",01.03.2020,10.00,133
2,35. Hördter Auwaldlauf,"76771 Hördt, Deutschland",01.03.2020,20.00,56
3,Flugplatzserie 2020,"67346 Speyer, Deutschland",28.02.2020,10.00,70
4,Flugplatzserie 2020,"67346 Speyer, Deutschland",24.01.2020,10.00,115
5,33. Oggersheimer Berglauf,"Oggersheim, 67071 Ludwigshafen am Rhein, Deuts...",16.02.2020,10.60,163
6,33. Oggersheimer Berglauf,"Oggersheim, 67071 Ludwigshafen am Rhein, Deuts...",16.02.2020,21.10,161
7,38. Winterlaufserie,"Rheinzabern, Deutschland",09.02.2020,10.00,118
8,38. Winterlaufserie,"Rheinzabern, Deutschland",09.02.2020,20.00,663
9,38. Winterlaufserie,"Rheinzabern, Deutschland",12.01.2020,5.00,119
